This notebook presupposes an OpenAI "clone" server running on localhost.

We used VLLM to create a local server using the Llama3 model. 

In particular, we used the following terminal command to initialize the server:

```python -m vllm.entrypoints.openai.api_server     --model meta-llama/Meta-Llama-3-8B-Instruct --dtype half ```

In [1]:
import pandas as pd
import openai
from pandas_parallel_apply import DataFrameParallel, SeriesParallel

In [2]:
sarcasm_headlines = pd.read_json('Sarcasm_Headlines_Dataset.json', lines=True)

In [3]:
sarcasm_headlines.head(5)

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [4]:
sarcasm_headlines['generated_sentence'] = "No Sentence"

In [8]:
def create_sentence(row):
    if row['generated_sentence'] == 'No Sentence':
        example = row['headline']
        model = "meta-llama/Meta-Llama-3-8B-Instruct"
        openai.api_base = "http://localhost:8000/v1"
        openai.api_key = "123"
        model_args={
                            "temperature": 0.7,
                            "max_tokens": 4000,
                            "top_p": 0.95,
                            "frequency_penalty": 0,
                            "presence_penalty": 0,
                        }
        if row['is_sarcastic'] == 1:
            messages = [{"role": "user", "content": f"You are a news headlines writer. Create a sarcastic headline similar to the example. Respond only with the headline. " + 
                     f"Here is an example sentence to help you. Example: {example}"}]
        else:
            messages = [{"role": "user", "content": f"You are a news headlines writer. Create a headline similar to the example. Respond only with the headline. " + 
                     f"Here is an example sentence to help you. Example: {example}"}]
        try:
            response = openai.ChatCompletion.create(
                        model=model, messages=messages, request_timeout=120, **model_args
                    )
            sentence = response['choices'][0]['message']['content']
            #print("Success:\n", sentence)
            return sentence
        
        except Exception as e:  # Exception
            print(f"Error: {e}.")
            if "response" in locals():
                print(f"Generated Sentence: {response['choices'][0]['message']['content']}")
    
            return "No Sentence"
    else:
       return row['generated_sentence']
    

In [9]:
sarcasm_headlines['generated_sentence'] = DataFrameParallel(sarcasm_headlines, n_cores = 20, pbar = True).apply(create_sentence, axis=1)

Process-22: 100%|██████████| 1431/1431 [15:59<00:00,  1.49it/s]


In [16]:
gen_sents = sarcasm_headlines[['generated_sentence','is_sarcastic']]
gen_sents.to_csv("sarcasm_headlines_synthetic_Llama_3_topp95_temp_7.csv")